In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('/content/Covid Toy Data.csv')
df.sample(5)


,age,gender,fever,cough,city,has_covid
92,82,Female,102.0,Strong,Kolkata,No
38,49,Female,101.0,Mild,Delhi,Yes
54,60,Female,99.0,Mild,Mumbai,Yes
57,49,Female,99.0,Strong,Bangalore,No
69,73,Female,103.0,Mild,Delhi,No


In [4]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [5]:
from numpy.random import random_sample
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns = ['has_covid']),df['has_covid'],test_size = 0.2)

In [6]:
X_test

,age,gender,fever,cough,city
46,19,Female,101.0,Mild,Mumbai
60,24,Female,102.0,Strong,Bangalore
8,19,Female,100.0,Strong,Bangalore
78,11,Male,100.0,Mild,Bangalore
52,47,Female,100.0,Strong,Bangalore
21,73,Male,98.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
91,38,Male,NaN,Mild,Delhi
37,55,Male,100.0,Mild,Kolkata
50,19,Male,101.0,Mild,Delhi


1) let's demonstrate how we preprocess data without using column transfomer

In [7]:
#adding simple imputer to fever column(cause it's missing 10 values)
si = SimpleImputer()
x_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
x_test_fever = si.fit_transform(X_test[['fever']])

x_train_fever.shape

(80, 1)

In [16]:
#OrdinalEncoding --> cough (the category types are dependent)
oe = OrdinalEncoder(categories = [['Mild','Strong']])
x_train_cough = oe.fit_transform(X_train[['cough']])

#also the test data
x_test_cough = oe.fit_transform(X_test[['cough']])

x_train_cough.shape

(80, 1)

In [9]:
#OneHotEncoding --> gender,city
ohe = OneHotEncoder(drop='first',sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

#also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [17]:
#Extracting age
x_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
x_train_age.shape


(80, 1)

In [20]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,X_train_gender_city,x_train_cough),axis=1)
# also the test data

x_test_transformed = np.concatenate((x_test_age,x_test_fever,X_test_gender_city,x_test_cough),axis=1)

x_train_transformed

array([[ 72.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 42.        , 101.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 25.        ,  99.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 42.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 64.        , 102.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 12.        ,  98.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 69.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 81.        , 101.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 66.        ,  99.        ,   1.        ,   0.        ,
          0.    

Using column transformer

In [19]:
from sklearn.compose import ColumnTransformer

In [27]:

transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [28]:
transformer.fit_transform(X_train)

array([[ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  25.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  42.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.    

In [29]:
transformer.transform(X_test)

array([[101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  19.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  24.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  19.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  11.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  73.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [100.90277778,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  38.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.    